In [123]:
import pandas as pd
with open('./coronavirusdataset/PatientInfo.csv', 'rb') as infile:
    patient_info = pd.read_csv(infile)
    
with open('./coronavirusdataset/PatientRoute.csv', 'rb') as infile:
    patient_route = pd.read_csv(infile)

In [124]:
len(patient_info.patient_id.unique())

3519

In [125]:
len(patient_route.patient_id.unique())

1319

In [126]:
data = pd.merge(patient_info, patient_route, on='patient_id')
data = data[data['province_x'] == 'Seoul']

In [127]:
data

,patient_id,global_num_x,sex,birth_year,age,country,province_x,city_x,disease,infection_case,...,released_date,deceased_date,state,global_num_y,date,province_y,city_y,type,latitude,longitude
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,...,2020-02-05,NaN,released,2.0,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632
1,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,...,2020-02-05,NaN,released,2.0,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,...,2020-03-02,NaN,released,5.0,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,...,2020-03-02,NaN,released,5.0,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,...,2020-03-02,NaN,released,5.0,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257,1000000711,10987.0,NaN,NaN,NaN,Korea,Seoul,Nowon-gu,NaN,overseas inflow,...,NaN,NaN,isolated,10987.0,2020-05-13,Seoul,Jungnang-gu,hospital,37.612772,127.098167
3258,1000000712,NaN,NaN,NaN,NaN,Korea,Seoul,Gangbuk-gu,NaN,NaN,...,NaN,NaN,isolated,NaN,2020-04-30,Seoul,Yeongdeungpo-gu,etc,37.539111,126.903394
3259,1000000712,NaN,NaN,NaN,NaN,Korea,Seoul,Gangbuk-gu,NaN,NaN,...,NaN,NaN,isolated,NaN,2020-05-09,Seoul,Yeongdeungpo-gu,beauty_salon,37.535056,126.899351
3260,1000000712,NaN,NaN,NaN,NaN,Korea,Seoul,Gangbuk-gu,NaN,NaN,...,NaN,NaN,isolated,NaN,2020-05-12,Seoul,Yeongdeungpo-gu,hospital,37.533121,126.903804


In [128]:
len(data.patient_id.unique())

640

In [129]:
pd.isna(data['infected_by']).sum()

2733

In [130]:
import networkx as nx

#instantiation of the directed graph
G = nx.Graph()
G.add_nodes_from(list(data['patient_id'].unique()))


for idx in data.index:
    if not(pd.isna(data['infected_by'][idx])):
        G.add_edge(data['patient_id'][idx],data['infected_by'][idx], weight=float(1))
        


len(G), G.size()

(644, 111)

In [131]:
## making groups of people who may have been infecting each other
groups = data.groupby(['latitude', 'longitude', 'type'])['patient_id'].unique().apply(list).tolist()

In [132]:
groups

[[1000000199],
 [1000000199],
 [1000000199],
 [1000000199],
 [1000000199],
 [1000000199],
 [1000000357, 1000000373],
 [1000000104],
 [1000000104],
 [1000000104],
 [1000000073],
 [1000000097],
 [1000000258],
 [1000000440],
 [1000000682],
 [1000000258],
 [1000000039, 1000000040],
 [1000000041],
 [1000000374, 1000000375],
 [1000000058],
 [1000000023],
 [1000000031],
 [1000000047],
 [1000000026,
  1000000044,
  1000000050,
  1000000056,
  1000000074,
  1000000099,
  1000000112,
  1000000123],
 [1000000047],
 [1000000050, 1000000058],
 [1000000112],
 [1000000474],
 [1000000474],
 [1000000215],
 [1000000231],
 [1000000081, 1000000184],
 [1000000077],
 [1000000235],
 [1000000174],
 [1000000038],
 [1000000038],
 [1000000543],
 [1000000489],
 [1000000108, 1000000206],
 [1000000650],
 [1000000035],
 [1000000489],
 [1000000598],
 [1000000038],
 [1000000122],
 [1000000148],
 [1000000093],
 [1000000037],
 [1000000009],
 [1000000187],
 [1000000187],
 [1000000009],
 [1000000009, 1000000255],
 [100000

In [143]:
for elt in groups:
    if len(elt) > 1:
        for i in range(len(elt)-1):
            G.add_edge(elt[i], elt[i++1], t=float(0.1))
            
len(G), G.size()

(644, 1170)

In [145]:
import numpy as np

char_path_length = 'not connected'
pos_max = np.argmax([len(listElem) for listElem in sorted(nx.connected_components(G))])
char_path_length_component = [nx.average_shortest_path_length(G.subgraph(g)) for g in sorted(nx.connected_components(G))][pos_max]

density = nx.density(G)

transitivity = nx.transitivity(G)
clustering_coef = nx.average_clustering(G)

n_connected_comp = nx.number_connected_components(G)
size_connected_comp = sum([len(listElem) for listElem in sorted(nx.connected_components(G))])/len(sorted(nx.connected_components(G)))

print(f'original characteristic path length {char_path_length}, \noriginal characteristic path length of biggest component {char_path_length_component},\noriginal network density {density},\noriginal transitivity {transitivity}, \noriginal average clustering coefficient {clustering_coef}, \noriginal number of connected components {n_connected_comp}, \noriginal average size of a connected component {size_connected_comp}')

original characteristic path length not connected, 
original characteristic path length of biggest component 5.6461423956740555,
original network density 0.005650918153453822,
original transitivity 0.06745737583395107, 
original average clustering coefficient 0.07180198166223016, 
original number of connected components 14, 
original average size of a connected component 46.0
